In [1]:
!pip install sacremoses 
!pip install rouge_score 
!pip install pycocoevalcap
# !pip install open_clip_torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 20.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=22454b22adab8e1ab24f434f4d0f07cacf3e4d84439d2e8dbfd6b74421285337
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.3/104.3 MB 16.7 MB/s eta 0:00:00


In [2]:
!git clone https://github.com/hein-nkhh/XrayImage_Gen_Report.git

Cloning into 'XrayImage_Gen_Report'...
remote: Enumerating objects: 475, done.
remote: Counting objects: 100% (122/122), done.
remote: Compressing objects: 100% (93/93), done.
remote: Total 475 (delta 54), reused 97 (delta 29), pack-reused 353 (from 1)
Receiving objects: 100% (475/475), 97.68 KiB | 6.51 MiB/s, done.
Resolving deltas: 100% (270/270), done.


In [3]:
%cd /kaggle/working/XrayImage_Gen_Report

/kaggle/working/XrayImage_Gen_Report


In [4]:
%%writefile /kaggle/working/XrayImage_Gen_Report/models/feature_extractos.py
import torch
from transformers import ViTModel, ViTFeatureExtractor
from config import DEVICE

class FeatureExtractor:
    def __init__(self):
        # Sử dụng CLIP-ViT thay cho Swin Transformer
        self.model = ViTModel.from_pretrained('google/vit-base-patch16-224-in21k').to(DEVICE)
        self.model.eval()
        self.processor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224-in21k')

    def extract(self, image):
        # Chuẩn bị ảnh và chú thích cho CLIP
        inputs = self.processor(images=image, return_tensors="pt")
        inputs = {k: v.to(DEVICE) for k, v in inputs.items()}
        with torch.no_grad():
            # Lấy các đặc trưng từ CLIP
            outputs = self.model.get_image_features(**inputs)
        return outputs[:, 0, :].squeeze(0).cpu().numpy()

Writing /kaggle/working/XrayImage_Gen_Report/models/feature_extractos.py


In [5]:
%%writefile /kaggle/working/XrayImage_Gen_Report/models/mlp.py
import torch
import torch.nn as nn
from config import MLP_HIDDEN_DIM

class MLP(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_dim, MLP_HIDDEN_DIM)
        self.dropout1 = nn.Dropout(0.1)
        self.fc2 = nn.Linear(MLP_HIDDEN_DIM, output_dim)
        self.layer_norm = nn.LayerNorm(output_dim)

    def forward(self, x):
        x = self.fc1(x)
        x = torch.relu(x)
        x = self.dropout1(x)
        x = self.fc2(x)
        x = self.layer_norm(x)
        return x


Overwriting /kaggle/working/XrayImage_Gen_Report/models/mlp.py


In [6]:
%%writefile /kaggle/working/XrayImage_Gen_Report/models/report_generator.py
# models/generator.py
import torch
from transformers import AutoTokenizer, T5ForConditionalGeneration
from config import BIOBART_MODEL_NAME, DEVICE

class ReportGenerator:
    def __init__(self):
        self.tokenizer = AutoTokenizer.from_pretrained('QizhiPei/biot5-base', model_max_length=512)
        self.model = T5ForConditionalGeneration.from_pretrained('QizhiPei/biot5-base').to(DEVICE)
        self.model.config.pad_token_id = self.tokenizer.pad_token_id

    def generate(self, embeddings, max_length=150):
        inputs = embeddings.unsqueeze(1)
        output_ids = self.model.generate(
            inputs_embeds=inputs,
            max_length=max_length,
            num_beams=4,
            early_stopping=True
        )
        return self.tokenizer.batch_decode(output_ids, skip_special_tokens=True)

    def get_loss(self, embeddings, labels, attention_mask):
        batch_size, seq_len = labels.shape
        inputs_embeds = embeddings.unsqueeze(1).expand(-1, seq_len, -1)
        vocab_size = self.model.config.vocab_size
        if labels.max() >= vocab_size:
            print("⚠️ Token ID vượt vocab size:", labels.max().item(), "vượt", vocab_size)
            print("Vị trí lỗi:", torch.nonzero(labels >= vocab_size))
            exit(1)

        output = self.model(
            inputs_embeds=inputs_embeds,
            labels=labels,
            decoder_attention_mask=attention_mask,
        )
        return output.loss


Overwriting /kaggle/working/XrayImage_Gen_Report/models/report_generator.py


In [7]:
!pwd

/kaggle/working/XrayImage_Gen_Report


In [8]:
!python main.py

2025-04-24 15:17:17.666103: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745507837.865620      73 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745507837.921489      73 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
🧪 Checking if checkpoint directory exists: /kaggle/working/checkpoint
📂 Created checkpoint directory: /kaggle/working/checkpoint
🔍 Extracting image features...
config.json: 100%|█████████████████████████| 71.8k/71.8k [00:00<00:00, 8.61MB/s]
model.safetensors: 100%|██████████████████████| 352M/352M [00:01<00:00, 287MB/s]
Extracting features: 100%|██████████████████████| 13/13 [00:39<00:00,  3.04s/it]
✅ Extracted features: (2750, 2048)

In [9]:
print("a")

a
